In [1]:
# import libraries
import pandas as pd
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import normalize
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import seaborn as sns

In [2]:
# making the files into dataframes
train = pd.read_csv('../data/kkbox-music-recommendation-challenge/train.csv')
song = pd.read_csv('../data/kkbox-music-recommendation-challenge/songs.csv')
members = pd.read_csv('../data/kkbox-music-recommendation-challenge/members.csv')
song_extra = pd.read_csv('../data/kkbox-music-recommendation-challenge/song_extra_info.csv')


# creating data time columns

members.registration_init_time = pd.to_datetime(members.registration_init_time, format='%Y%m%d', errors='ignore')
members['registration_init_time_year'] = members['registration_init_time'].dt.year.astype('int64')
members['registration_init_time_month'] = members['registration_init_time'].dt.month.astype('int64')
members['registration_init_time_day'] = members['registration_init_time'].dt.day.astype('int64')
members.expiration_date = pd.to_datetime(members.expiration_date,  format='%Y%m%d', errors='ignore')
members['expiration_date_year'] = members['expiration_date'].dt.year.astype('int64')
members['expiration_date_month'] = members['expiration_date'].dt.month.astype('int64')
members['expiration_date_day'] = members['expiration_date'].dt.day.astype('int64')
members = members.drop(columns=['registration_init_time','expiration_date'])

# merging tables together to one complete dataframe
train_members = pd.merge(train, members, on='msno', how='inner')
train_merged = pd.merge(train_members, song, on='song_id', how='outer')
train_merged = pd.merge(train_merged, song_extra, on='song_id', how='left')

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
train_merged['msno'] = train_merged['msno'].astype(str)
train_merged['msno'] = labelencoder.fit_transform(train_merged['msno'])


In [3]:
# More Data Cleaning
# Find the number of songs each user has listened once and more than once
SongsPerUser = train_merged.msno.value_counts()

# selecting only the songs that are listened by more than 10 users
train_merged = train_merged[train_merged["msno"].isin(SongsPerUser[SongsPerUser>10].index)]

# making matrix for of all songs listened 
userItemRatingMatrix=pd.pivot_table(train_merged, values='target',
                                    index=['msno'], columns=['name'])
#  replacing all the 0 values with null value
userItemRatingMatrix = userItemRatingMatrix.replace(0,np.nan)




In [4]:
# To Find the k nearest neighbours of active user first find the distance of active user to all other users

from scipy.spatial.distance import hamming
def nearestneighbours(userx,K=3):
    
    # create a user df that contains all users except active user
    allUsers = pd.DataFrame(userItemRatingMatrix.index)
    allUsers = allUsers[allUsers.msno!=userx]
    
    # Add a column to this df which contains distance of active user to each user
    allUsers["distance"] = allUsers["msno"].apply(lambda x: hamming(userItemRatingMatrix.loc[userx],userItemRatingMatrix.loc[x]))
    KnearestUsers = allUsers.sort_values(["distance"],ascending=True)[["msno","distance"]][:K]
    
    
    return KnearestUsers


In [5]:
userItemRatingMatrix

name,24karats –type S-(與放浪兄弟EXILE、DOBELMAN合作超跳嘻哈單曲),A Gentle Dissolve (Shawn Lee Ping Pong Orchestra Remix)(Thievery Corporation),A-CHA,Adicto,Be Your Sound (Radio Edit),Besame mucho (吻我，深深地吻我吧with 克里斯伯堤),Dont Cry for Me Argentina [Evita],Draw Me Close To You,Employee Of The Month (Album Version),Everytime We Touch – DJ Jerry feat. Missy Babe,...,ｴﾀﾆﾃｨ (Eternity),ｸﾞﾗｲﾀﾞｰ (Glider (rmx ver. /Live edit)),ｼｭｶﾞｰﾚｽｶﾞｰﾙ (Sugarless GiRL),ｼﾞｪﾘｰｱｰﾙｴﾑｴｯｸｽﾊﾞｰｼﾞｮﾝ (Jelly (rmx ver.)),ｼﾞｬﾝﾊﾟｰﾗｲﾌﾞﾐｯｸｽ (JUMPER (Live mix)),ﾄﾞﾘｰﾐﾝﾄﾞﾘｰﾐﾝ (dreamin dreamin),ﾌﾗｯｼｭﾊﾞｯｸ (FLASH BACK (Extended-Live mix)),ﾌﾙｰﾂｸﾘｯﾊﾟｰ (FRUITS CLiPPER),ﾓｱﾓｱﾓｱ (more more more),ﾚﾄﾛﾒﾓﾘｰ (Retro Memory)
msno,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# function that allow inputting new user and return recommended songs

# input vairables:
#  - input a new custom user id(new_user_id)
#  - a list of songs of ur choice(songs)
#  - and the number of songs that will be recommended(N, the default number is 10)

def new_user(new_user_id,songs,N=10):
    
    # calling the global variable userItemRatingMatrix
    global userItemRatingMatrix
    
    # drop the user row if the input user id is already existed
    if new_user_id in userItemRatingMatrix.index:
        userItemRatingMatrix = userItemRatingMatrix.drop([new_user_id])
        
    # append a new empty row with the index of newly input user id    
    userItemRatingMatrix = userItemRatingMatrix.append(pd.Series(name=new_user_id))
    
    # for each song in the input song list
    # search the input song in the userItemRatingMatrix and change the NaN value into 1 on the new user row
    for i in songs:
        for j in userItemRatingMatrix.columns:
            if i == j:
                userItemRatingMatrix.loc[new_user_id,j] = 1
                
    # find the nearest 50 neighbours based on the newly input user song info            
    KnearestUsers = nearestneighbours(new_user_id,50)  
    
    # generate a df that contains all song info of the 50 neighbours
    NNRatings = userItemRatingMatrix[userItemRatingMatrix.index.isin(KnearestUsers.index)]
    
    # create an list with the same length of number of neighbours containing with the same values (1)
    emp_list= []
    
    for i in range(len(NNRatings.index)):
        emp_list.append(1)   
    
    
    # find the no. appearance of the songs among the 50 neighbours
    
    NNRatings.index = emp_list
    temp  = NNRatings.groupby(NNRatings.index).sum()
    avgRating = temp.loc[1].sort_values(ascending=False)
    
    # extract the songs from the input playlist that are also existing in the matrix
    songsListened = userItemRatingMatrix.loc[new_user_id].dropna().index
    
    # return the recommended songs in descending order according to the frequency
    avgRating = avgRating[~avgRating.index.isin(songsListened)]
    topN_songs = avgRating.sort_values(ascending=False)[:N]
    
    
    return topN_songs
                

In [ ]:
gemma =['Bugatti','Monster','Congratulations','Look At Me Now','m.A.A.d city','Famous','Trap Queen','23','Clique','HUMBLE.',"Hustlin'",'Wild for the Night','Black Beatles','White Iverson','Panda','DNA.','All Of The Lights ','Bound 2','Amazing','Heartless']

new_user(-1,gemma)

In [ ]:
jacky =["Saturn", "Star Star", "C'mon Through", "Faded", "Canon in D Major, P.37", "Me Before You Orchestral" , "Say Something", "Spirit Walk", "No Surprises", "The Rain", "The Ludlows", "The Blower's Daughter", "Wake Me up When September Ends"]
new_user(-2,jacky)


In [ ]:
songs = ["Tik Tok","Need You Now","Hey, Soul Sister""CaliforniaGurls""OMG","Airplanes",
        "Love the Way You Lie","Bad Romance","Dynamite","Break Your Heart",
        "Nothin' on You","I Like It","BedRock","In My Head","Rude Boy",
        "Telephone","Teenage Dream","Just the Way You Are","Cooler Than Me","Imma Be",
        "Empire State of Mind","DJ Got Us Fallin' in Love","Billionaire"
        ,"Not Afraid","Replay","Sexy Bitch","Breakeven","Your Love Is My Drug",
        "I Gotta Feeling","Fireflies","Say Aah","Find Your Love","Alejandro"
        ,"Ridin' Solo","Just a Dream","How Low","Like a G6","Carry Out","Haven't Met You Yet",
        "Club Can't Handle Me","Down","Bulletproof","Whatcha Say","Baby","Whataya Want from Me"
        ,"Mine","Only Girl (In the World)","Live Like We're Dying","Hard","Young Forever","Blah Blah Blah"
        ,"Bottoms Up","Do You Remember","All the Right Moves","According to You","My Chick Bad",
        "You Belong with Me","Meet Me Halfway","Take It Off","Over","Animal","Misery","Magic","Paparazzi"
        ,"Tie Me Down","Your Love","Party in the U.S.A.","Deuces","3","Impossible","Forever"
        ,"Two Is Better Than One","My First Kiss","Already Gone","Rock That Body","Secrets"
        ,"Naturally","Un-Thinkable (I'm Ready)","All I Do Is Win","I Made It (Cash Money Heroes)"
        ,"Stuck Like Glue","Hey Daddy (Daddy's Home)","There Goes My Baby","Today Was a Fairytale"
        ,"Say Something","Sweet Dreams","Use Somebody","Undo It","Eenie Meenie","Right Above It"
        ,"The House That Built Me","If I Die Young","The Only Exception","American Honey"
        ,"King of Anything","Life After You","Smile","Teach Me How to Dougie","Try Sleeping with a Broken Heart"
        ,"Lover, Lover"]
new_user(-3,songs)

In [9]:
songs = ["Tik Tok","Need You Now","Hey, Soul Sister","CaliforniaGurls","OMG","Airplanes",
        "Love the Way You Lie","Bad Romance","Dynamite","Break Your Heart",
        "Nothin' on You","I Like It","BedRock","In My Head","Rude Boy",
        "Telephone","Teenage Dream","Just the Way You Are","Cooler Than Me","Imma Be",
        "Empire State of Mind","DJ Got Us Fallin' in Love","Billionaire"
        ,"Not Afraid","Replay","Sexy Bitch","Breakeven","Your Love Is My Drug",
        "I Gotta Feeling","Fireflies","Say Aah","Find Your Love","Alejandro"
        ,"Ridin' Solo","Just a Dream","How Low","Like a G6","Carry Out","Haven't Met You Yet",
        "Club Can't Handle Me","Down","Bulletproof","Whatcha Say","Baby","Whataya Want from Me"
        ,"Mine","Only Girl (In the World)","Live Like We're Dying","Hard","Young Forever","Blah Blah Blah"
        ,"Bottoms Up","Do You Remember","All the Right Moves","According to You","My Chick Bad",
        "You Belong with Me","Meet Me Halfway","Take It Off","Over","Animal","Misery","Magic","Paparazzi"
        ,"Tie Me Down","Your Love","Party in the U.S.A.","Deuces","3","Impossible","Forever"
        ,"Two Is Better Than One","My First Kiss","Already Gone","Rock That Body","Secrets"
        ,"Naturally","Un-Thinkable (I'm Ready)","All I Do Is Win","I Made It (Cash Money Heroes)"
        ,"Stuck Like Glue","Hey Daddy (Daddy's Home)","There Goes My Baby","Today Was a Fairytale"
        ,"Say Something","Sweet Dreams","Use Somebody","Undo It","Eenie Meenie","Right Above It"
        ,"The House That Built Me","If I Die Young","The Only Exception","American Honey"
        ,"King of Anything","Life After You","Smile","Teach Me How to Dougie","Try Sleeping with a Broken Heart"
        ,"Lover, Lover"]


In [ ]:
train_merged.to_excel('train_merged.xlsx')

In [ ]:
train_merged

In [ ]:
a4_dims = (20, 9)
fig, ax = plt.subplots(figsize=a4_dims)

plt.title('Heatm...ap of KKBOX Dataset',fontsize=20)
ax=sns.heatmap(data=train_merged.corr(),annot=True,fmt=".2f")

In [ ]:


a4_dims = (15, 7)
fig, ax = plt.subplots(figsize=a4_dims)
ax=sns.countplot(x="gender",data=train_merged,palette=['#705898','#7038F8','yellow'],hue="target")
plt.xlabel("male female participation")
plt.ylabel("count")
plt.title("male female participation vs target ")
plt.xticks(rotation='90')
plt.legend(loc='upper left')
plt.show()



In [ ]:
plt.figure(figsize=(15,10))
sns.set(font_scale=1.5)
sns.countplot(x='source_system_tab',hue='source_system_tab',data=train,dodge=False)
sns.set(style="darkgrid")
plt.xlabel('source types',fontsize=20)
plt.ylabel('count',fontsize=20)
plt.xticks(rotation='45')
plt.title('Frequency of System Tab Used',fontsize=20)
plt.tight_layout()


In [ ]:
import matplotlib as mpl

mpl.rcParams['font.size'] = 20
labels = ['Male','Female']
plt.figure(figsize = (9, 9))
sizes = pd.value_counts(members.gender)
patches, texts, autotexts = plt.pie(sizes, 
                                    labels=labels, autopct='%.0f%%',
                                    shadow=False, radius=1,startangle=90)
for t in texts:
    t.set_size('smaller')
plt.legend()
plt.show()

In [10]:
def evaluation_metrics_god(userid,x):
    list_recom = new_user(userid,x,30).index
    list_remaining_songs = x[10:]
    total_songs = len(set(list_remaining_songs).intersection(song_matrix.columns))
    matched_songs = len(list_recom.index.intersection(list_remaining_songs))
    print(list_recom.index)

    matched_rate = matched_songs/total_songs

    return matched_rate, total_songs, matched_songs


In [11]:
evaluation_metrics_god(-10,songs)

<ipython-input-6-90b9630bcea9>:18: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  userItemRatingMatrix = userItemRatingMatrix.append(pd.Series(name=new_user_id))


NameError: name 'song_matrix' is not defined